# Primeiros passos com openMP
## 1.Aplicação
Resolução de problemas de maneira paralelizada.
Resolução de problemas com independencia de dados, bem como aceleração de problemas já existentes. 
Podemos medir seu desempenho pelo tempo de speed-up: 
- tempo de resposta 
- speed-up/processador
- Eficiencia Energética: Mflops/Watt


## 2. Primeiro contato
Programação com Threads
Suporte em multiplas plataformas e linguagens, com diferentes abstrações 
No nivel mais baixo, provido pelo SO, há primitivas para criar threads. 

API tradicional - Posix Threads(pthreads)

### Modelo de programação
- Criação de um time de threads para executar um bloco de código paralelo. 
- Time de threads pode replicar o codigo em paralelo, ou dividi-lo entre threads. 
- Threads podem ser criadas dinamicamente
- Flexibilidade em definir como variaveis do programa serão `compartilhadas` ou `replicadas` entre threads. 




*Lembre-se*:

- SIMD(Single Instruction Multiple Data) = Uma instrução executada para multiplos dados, de modo que a mesma instrução de uma unidade de controle e então executa em multiplo e diferentes dataSets. Isso permite que performe operações identicas em diferentes operandos. SIMD permite que 4, 8... 512 ? bits sejam executados no mesmo ciclo de clock. Vale lembrar tambem que SIMD usa `execution units` e nao `threads` ou `cores`.

- SIMT(Single Instruction Multiple Threads) = Bem similar com SIMD, com a diferença que SIMT faz o uso de threads, em diferentes data-set's, reduzindo a latencia que vem com busca primaria de busca. 

- SMT(Simultaneous Multi-Threading) = Permite o core da CPU rodar multiplas threads ao mesmo tempo.  Teóricamente, voce pode ter 8 threads por core via SMT, mas é viável ter somente 2.

### Hello Word from openMP

In [ ]:
%%writefile src/01-intro.c
#include<stdio.h>
#include<omp.h>

int main(){

  #pragma omp parallel
  { // Inicio Paralelização
    int ID = omp_get_thread_num();
    printf("Hello thread %d  ", ID);
    printf("Same thread %d\n", ID);

  } // Final paralelização
}


In [ ]:
!gcc -fopenmp src/01-intro.c -o src/01-intro; ./src/01-intro

Observe o resultado, não necessariamente o ID do thread estará alinhado com a sequencia crescente. Isso ocorre porque as threads podem ser executadas concorrentemente. e em diferentes processadores, caso existam.

### Visao Geral de OpenMP
- Modelo de multithreading de memória compartilhada. 
- Compartilhamento nao intencional de dado, que causa condições de corrida. 
- O programa de memória Compartilhada: instancia do programa -- um processo e muitas threads. Threads interagem através de leituras/escritas com o espaço de endereçamento compartilhado. 
- Escalonador SO decide quando executar cada thread(entrelaçado para ser justo). 
- Thread é basicamente uma pilha com uma parte do processo a ser executado, lembrando que variaveis globais/dinamicas podem ser acessadas por qualquer thread. 
- Threads podem ser criadas e destruidas ao longo da execução do programa, lembrando que a sincronização garante a ordem.


### Criação de Threads: Regiões Paralelas


In [ ]:
%%writefile src/02-array.c
#include<stdio.h>
#include<omp.h>

void fill(int pos, float val, float* A, int N)
{
  A[pos] = val;
  printf("A[%d] = %.4f\n", pos, val);
}

const int N = 1000;
int main(){

  float A[N]; 
  #pragma omp parallel num_threads(4)
  { 
    int ID = omp_get_thread_num();
    fill(ID, 10.8, A, N);

    

  } 
}


Execute a célula abaixo e verifique a sequencia dos indices do vetor assinalado, provavelmente não será uma saída sequencial. Porque?

In [ ]:
!gcc -fopenmp src/02-array.c -o src/02-array; ./src/02-array

### Usando Laços em openMP

In [ ]:
%%writefile src/03-for_OMP.c
#include<stdio.h>
#include<omp.h>

const int N = 1000;
int main(){
  int A[N]; 
  int B[N]; 
  // Fill vectors
  for (int i = 0; i < N ; i++) 
  {
    A[i] = 1;
    B[i] = 10; 
  }
  
  double start = omp_get_wtime();
  /*Abre um pool para "x" threads, e dividimos o trabalho entre si
    Nesse caso, teremos 4 "caminhos" que executam uma parte da soma vetorial
  */
  #pragma omp parallel
  {
    int id, i, n_threads, i_start, i_end; 
    id = omp_get_thread_num();
    n_threads = omp_get_num_threads(); 
    
    //divisao de elementos pelo numero de threads
    // i_start =  id    * N / n_threads ; 
    //i_end   = (id+1) * N / n_threads ;

    //impede de acessar memoria indevida
    //if(id == n_threads -1) i_end = N;
    #pragma omp parallel for
    for( i=0; i < N; i++){
      A[i] = A[i] + B[i];
    }     
  }


  double end = omp_get_wtime();
  printf("Parallel Time: %.4lf ms\n", (end-start)*1e3);
  return 0;
}


In [ ]:
%%writefile src/03-for_serial.c
#include<stdio.h>
#include<omp.h>
#include <sys/time.h>
const int N = 1000;
int main()
{
  int A[N];
  int B[N];
  struct timeval start;
  struct timeval end;
  // Fill vectors
  
  for (int i = 0; i < N ; i++) {
    A[i] = 1;
    B[i] = 10; 
  }

  gettimeofday(&start, NULL);
  //start = time(NULL); //omp_get_wtime();
  for(int i = 0; i < N; i++)
    A[i] = A[i] + B[i];
  gettimeofday(&end, NULL);
  //end = time(NULL); // omp_get_wtime();
  double t_time = end.tv_sec*1000000 + end.tv_usec - start.tv_sec*1000000 + start.tv_usec;
  printf("Serial Time: %lf ms\n", t_time*1e-6);
  return 0;
}


In [ ]:
!gcc -fopenmp src/03-for_OMP.c -o src/03-for_OMP; ./src/03-for_OMP
#timeit faz uso do tempo da CPU -> use omp_get_wtime();

In [ ]:
!gcc src/03-for_serial.c -o src/03-for_serial; ./src/03-for_serial

## Aplicação Prática de Multiplicação

Abaixo temos um exemplo de multiplicação matricial com openMP.
Neste exemplo, faremos uso das seguintes diretivas: 
- shared: variavel disponível para leitura em todas as threads.
- private: é uma diretiva que informa pro compilador uma lista de variaveis que serao instanciadas para cada thread, na regiao em paralelo. O tipo de cada variavel privada é determinado pelo contexto.

In [ ]:
%%writefile src/03-matmul.c
#include<stdio.h>
#include<omp.h>
#include<time.h>

#define N 500

void matrix_mul(int A[N][N], int B[N][N], int C[N][N])
{
  int i, j, k;
  for ( i = 0; i < N; i++){
    for (j = 0; j < N; j++){
      for (k = 0; k < N; k++)
        C[i][j] += A[i][k] * B[k][j]; 
    }
  }
}

void matrix_mul_OMP(int A[N][N], int B[N][N], int C[N][N])
{
  int i, j, k;
  #pragma omp parallel for private(i, j, k) shared(A,B,C)
  for ( i = 0; i < N; i++){
    for (j = 0; j < N; j++){
      for (k = 0; k < N; k++)
        C[i][j] += A[i][k] * B[k][j]; 
    }
  }

}

int main(){
  
  double start_s, end_s, start_p, end_p;
  int i, j;
  int A[N][N]; 
  int B[N][N]; 
  int C_OMP[N][N];
  int C_serial[N][N];
  // Fill vectors
  for (i = 0; i < N ; i++) {
    for (j = 0; j < N ; j++) {
      A[i][j] = i*i;
      B[i][j] = -i; 
    }
  }
  
  start_s = omp_get_wtime();
  matrix_mul(A, B, C_serial);
  end_s = omp_get_wtime();
  
  
  printf("Serial Time:   %.4lf ms\n", end_s-start_s);

  start_p = omp_get_wtime();
  matrix_mul_OMP(A, B, C_OMP);
  end_p = omp_get_wtime();
  
  printf("Parallel Time: %.4lf ms\n", end_p-start_p);
  printf("Speed-up :     %.4lf ", (end_s-start_s)/(end_p-start_p));
  return 0;
}

In [36]:
!gcc -fopenmp src/03-matmul.c -o src/03-matmul; ./src/03-matmul

Serial Time:   0.7180 ms
Parallel Time: 0.4003 ms
Speed-up :     1.7935 

# Distribuindo de tarefas em outros *devices*
Com a nova atualização de **openMP 4.0**, foram adicionadas novas diretivas em C++ que extende o uso não só para CPU's, mas também para GPU's. Os compiladores intel permitem o uso desta nova atualização. 

1. Diretiva `target`: Esta diretiva permite o direcionamento do tipo de *device* para um padrão específico de arquitetura: GPU's ou CPU's.

2. Diretiva `map`: usada para direcionar os dados que, associada com outras clausulas, informa o método de como o dado será enviado para o *device*
  - **to**:   aloca o dado e move o dado para o *device*
  - **from**: aloca o dado e move do *device*.
  - **tofrom**: aloca o dado e move para e do *device*.
  - **alloc**: aloca dado no *device* (memória).
  - **delete**: deleta o dado alocado no *device*.

## openMP Offload na prática
No exemplo a seguir,será feito uma multiplicação  vetorial. Observe as *flags* utilizadas para envio dos dados para o device.

Vale destacar que caso esteja executando este *notebook* em máquina local, a execução da célula abaixo resultará em erro caso não haja GPU.


In [37]:
%%writefile src/04-vec_mul-GPU.c

#include <stdio.h> 
#include <stdlib.h>
#include <omp.h>

const int N = 10;
int main() {
  int *data = (int*)malloc(N * sizeof(int));
  int is_cpu;
  int scal = 20;
  //preenchimento do vetor
  for(int i=0; i < N; i++)
    data[i] = i;
  
  //Definição dos dados e sua maneira de envio
  
  #pragma omp target map(from : is_cpu)\
                     map(tofrom : data [0:N])\
                     map(from: scal)
  
  {
    //uma flag do openMP - verifica se ainda estamos no host
    is_cpu = omp_is_initial_device();
    
    //execução em paralelo 
    #pragma omp parallel for
    for (int i = 0; i < N; i++) {
      data[i] *= scal;
    }
  }
  
  for(int i=0; i < N; i++)
    printf("%d \n", data[i]);
  


  return 0;
}

Overwriting src/04-vec_mul-GPU.c


In [38]:
!gcc -fopenmp src/04-vec_mul-GPU.c -o src/04-vec_mul-GPU; ./src/04-vec_mul-GPU

src/04-vec_mul-GPU.c: In function ‘main._omp_fn.0.hsa.0’:
cc1: warning: could not emit HSAIL for the function [-Whsa]
cc1: note: support for HSA does not implement non-gridified OpenMP parallel constructs.
lto-wrapper: fatal error: could not find accel/nvptx-none/mkoffload in /usr/lib/gcc/x86_64-linux-gnu/9/:/usr/lib/gcc/x86_64-linux-gnu/9/:/usr/lib/gcc/x86_64-linux-gnu/:/usr/lib/gcc/x86_64-linux-gnu/9/:/usr/lib/gcc/x86_64-linux-gnu/ (consider using ‘-B’)

compilation terminated.
/usr/bin/ld: error: lto-wrapper failed
collect2: error: ld returned 1 exit status
/bin/bash: ./src/04-vec_mul-GPU: No such file or directory


## Referencias Bibliográficas
